In [5]:
%pip install bs4
%pip install selenium
%pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup, NavigableString
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.options import Options

In [7]:
base_url = 'https://www.saatva.com'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
}

In [8]:

def get_product_soup(product_type):    
    product_url = f'{base_url}/{product_type}'
    product_response = requests.get(product_url, headers=headers)
    print(product_response)
    return BeautifulSoup(product_response.content,'html.parser')
   

In [9]:
def get_price_range(soup):
    price_divs = soup.find_all('div', class_='detailProductTile__price')

    # Extract the desired price range from each div
    price_ranges = []
    for div in price_divs:
        price = ''.join([child.string for child in div.children if isinstance(child, NavigableString)])
        price_ranges.append(price.strip())

    return price_ranges

In [10]:
def get_product_links(soup, product_type):
  product_divs = soup.find_all('div', class_='detailProductTile__contentRow')
  product_links = []
  for div in product_divs:
    a_tag = div.find('a')
    if a_tag:
      href_value = a_tag['href']
      if href_value.startswith(f'/{product_type}'):
        product_links.append(f'{base_url}{href_value}')

  return product_links

In [11]:
def get_title(soup):
  try: 
    title = soup.find('h1', class_='productPanel__headingTitle').text.strip()

  except AttributeError:
    title = soup.find('h1', class_='dataStreamProductTitle__headingTitle').text.strip()

  return title

In [43]:
def get_short_description(soup):
    description_element = soup.find('p', class_='productPanel__shortDescription--mattressPDP')

# Extract the text from the located element
    try:
        description_text = description_element.text
    except:
        description_text = None

    if description_text is None:
        try:
            description_element = soup.select_one('.productPanel__shortDescription.productPanel__shortDescription--mattressPDP')
            description_text = description_element.text
        except:
            description_text = None

    if description_text is None:
        try:
            description_element = soup.find('p', class_='productPanel__shortDescription')
            description_text = description_element.text
        except:
            description_text = None

    if description_text is None:
        try:
            description_element = soup.find('p', class_='dataStreamProductTitle__shortDescription')
            description_text = description_element.text
        except:
            description_text = None
          
    return description_text

In [44]:
def get_long_description(soup):
    div_element = soup.find('div', class_='productPanel__description u-marginTop--xs')
    if div_element is not None:
        try:
            main_description = div_element.find('p').text
        except:
            main_description = None
    else:
        main_description = None
    
    if main_description is None:
        try:
            div = soup.find('div', class_='productPanel__description')
            main_description = div.find('p', recursive=False).text
        except:
            main_description = None
    
    if main_description is None:
        try:
            div_element = soup.find('div', class_='dataStreamLongDescription')
            main_description = div_element.find_all('p')[1].text
        except:
            main_description = None
   
    return main_description

In [19]:
def get_price(soup):
  try: 
    price_text = soup.find('span', class_='smallPriceDisplay__finalPrice').text
    if price_text == "":
        price = None
    else:
        price = int(''.join(filter(str.isdigit, price_text)))
  except AttributeError:
    price_text = soup.find('span', class_='dataStreamSmallPriceDisplay__finalPrice').text
    if price_text == "":
        price = None
    else:
        price = int(''.join(filter(str.isdigit, price_text)))

  return price

In [20]:
def get_sizes(soup):
  return [div.get_text() for div in soup.select('.productPanel__toggles--size .formToggle')]

In [21]:
def get_comfort_levels(soup):
  return [div.get_text().strip() for div in soup.select('div[data-selector="buystackToggle--comfort"]')]

In [22]:
def get_details(soup):
  value_prop_cards = soup.find_all(class_='valuePropCard__content')

  info_list = []

  for card in value_prop_cards:
    title = card.find(class_='valuePropCard__content-heading-title').text
    description_items = card.find_all(class_='valuePropCard__listItem')
    description = [item.text for item in description_items]

    info_list.append({
        "Attribute": title,
        "Description": description
    })

  return info_list

In [35]:
def get_ratings_reviews(url):

    options = Options()
    options.add_argument('--headless=new')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.get(url)
    driver.implicitly_wait(10)  # wait for up to 10 seconds for elements to be available 
    
    # Find the element containing the rating
    try:
        rating_element = driver.find_element(By.CLASS_NAME, 'sr-only')
        rating_text = rating_element.text
    except:
        rating_text = ""

    if rating_text == "":
        try:
            rating_element = driver.find_element(By.CLASS_NAME, 'avg-score')
            rating_text = rating_element.text
        except:
            rating_text = ""

    if rating_text == "":
        try:
            rating_element = driver.find_element(By.XPATH, '//span[@class="avg-score font-color-gray-darker"]')
            rating_text = rating_element.text
        except:
            rating_text = ""


    # Find the element containing the number of reviews
    try:
        reviews_element = driver.find_element(By.XPATH, '//a[@class="text-m" and contains(@aria-label, "reviews")]')
        reviews_text = reviews_element.text
    except:
        reviews_text = ""

    if reviews_text == "":
        try:
            reviews_element = driver.find_element(By.CLASS_NAME, 'reviews-qa-label')
            reviews_text = reviews_element.text
        except:
            reviews_text = ""

    if reviews_text == "":
        try:
            reviews_element = driver.find_element(By.XPATH, '//span[@class="reviews-qa-label font-color-gray"]')
            reviews_text = reviews_element.text.split()[0] 
        except:
            reviews_text = ""

    #Turn the text into a float/int
    if rating_text == "":
        rating = None
    else:
        rating = float(re.search(r"(\d+\.\d+)", rating_text).group(1))

    if reviews_text == "":
        reviews = 0
    else:
        reviews = int(re.search(r"(\d+)", reviews_text).group(1))

    return (rating, reviews)

In [24]:
#If this times out, run all product types individually

d = {"Product":[],"Description":[], "Price in Dollars":[], "Price Range":[], "Sizes":[],"Comfort Levels":[], "Details":[],
"Rating":[], "Number of Reviews":[]}

product_types = ('mattresses', 'bedding', 'furniture')
    

for product_type in product_types:
    product_soup = get_product_soup(product_type)
    product_links = get_product_links(product_soup, product_type)
    price_ranges = get_price_range(product_soup)
    for i, link in enumerate(product_links):
        pd_page = requests.get(link, headers=headers)
        pd_soup = BeautifulSoup(pd_page.content,'html.parser')
        d['Product'].append(get_title(pd_soup))
        d['Description'].append(get_short_description(pd_soup))
        d['Price in Dollars'].append(get_price(pd_soup))
        d['Price Range'].append(price_ranges[i])
        d['Sizes'].append(get_sizes(pd_soup))
        d['Comfort Levels'].append(get_comfort_levels(pd_soup))
        d['Details'].append(get_details(pd_soup))
        rating, reviews = get_ratings_reviews(link)
        d['Rating'].append(rating)
        d['Number of Reviews'].append(reviews)

In [ ]:
saatva_df = pd.DataFrame.from_dict(d)
saatva_df.to_csv("../data/saatva_data.csv", header=True, index=False)

In [57]:
def for_single_product(product_type, d):
    product_soup = get_product_soup(product_type)
    product_links = get_product_links(product_soup, product_type)
    price_ranges = get_price_range(product_soup)
    for i, link in enumerate(product_links):
        pd_page = requests.get(link, headers=headers)
        pd_soup = BeautifulSoup(pd_page.content,'html.parser')
        d['Product'].append(get_title(pd_soup))
        d['Description'].append(get_short_description(pd_soup))
        d['Price in Dollars'].append(get_price(pd_soup))
        d['Price Range'].append(price_ranges[i])
        d['Sizes'].append(get_sizes(pd_soup))
        d['Comfort Levels'].append(get_comfort_levels(pd_soup))
        d['Details'].append(get_details(pd_soup))
        rating, reviews = get_ratings_reviews(link)
        d['Rating'].append(rating)
        d['Number of Reviews'].append(reviews)

In [47]:
d_mattress = {"Product":[], "Description":[],"Price in Dollars":[], "Price Range":[], "Sizes":[],"Comfort Levels":[], "Details":[],
"Rating":[], "Number of Reviews":[]}
for_single_product('mattresses', d=d_mattress)

<Response [200]>


In [ ]:
saatva_mattress_df = pd.DataFrame.from_dict(d_mattress)
saatva_mattress_df.to_csv("../data/saatva_mattress_data.csv", header=True, index=False)

In [58]:
d_furniture = {"Product":[], "Description":[],"Price in Dollars":[], "Price Range":[], "Sizes":[],"Comfort Levels":[], "Details":[],
"Rating":[], "Number of Reviews":[]}
for_single_product('furniture', d=d_furniture)

<Response [200]>


In [59]:
saatva_furniture_df = pd.DataFrame.from_dict(d_furniture)
saatva_furniture_df.to_csv("../data/saatva_furniture_data.csv", header=True, index=False)

In [61]:
d_bedding = {"Product":[], "Description":[],"Price in Dollars":[], "Price Range":[], "Sizes":[],"Comfort Levels":[], "Details":[],
"Rating":[], "Number of Reviews":[]}
for_single_product('bedding', d=d_bedding)

<Response [200]>


In [ ]:
saatva_bedding_df = pd.DataFrame.from_dict(d_bedding)
saatva_bedding_df.to_csv("../data/saatva_bedding_data.csv", header=True, index=False)